In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from tqdm.auto import tqdm

# Подготовка датасета

In [3]:
#импортируем датасет

df = pd.read_csv('wiki_movie_plots_deduped.csv', sep=',')

In [4]:
df.head(5)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [5]:
#приводим к нижнему регистру

df['Genre_1'] = df['Genre'].str.lower()
df['Plot'] = df['Plot'].str.lower()

In [6]:
# количество фильмов по жанрам
df['Genre_1'].value_counts()

unknown                          6083
drama                            5964
comedy                           4379
horror                           1167
action                           1098
                                 ... 
cbc-tv miniseries                   1
bio-drama                           1
national film board docudrama       1
cult drama                          1
horror romantic comedy              1
Name: Genre_1, Length: 2265, dtype: int64

In [7]:
# в датасете 2265 жанров, необходимо преобразовать данные для предсказания жанров
# тестирую на копии df

df_copy = df.copy()

In [8]:
#преобразуем данные,чтобы разбить жанры по словам

df_copy['Genre_2']=df_copy['Genre_1'].str.replace(',', '/')
df_copy['Genre_2']=df_copy['Genre_2'].str.replace(' / ', '/')
df_copy['Genre_2']=df_copy['Genre_2'].str.replace('/ ', '/')
df_copy['Genre_2']=df_copy['Genre_2'].str.replace(' /', '/')
df_copy['Genre_2']=df_copy['Genre_2'].str.replace(' ', '/')
df_copy['Genre_2']=df_copy['Genre_2'].str.replace('/&/', '/')
df_copy['Genre_2']=df_copy['Genre_2'].str.replace('/-/', '/')
df_copy['Genre_2']=df_copy['Genre_2'].str.replace('sci-fi', 'sci_fi')
df_copy['Genre_2']=df_copy['Genre_2'].str.replace('bio-pic', 'biopic')
df_copy['Genre_2']=df_copy['Genre_2'].str.replace('-', '/')

In [9]:
df_copy.sample(5)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre_1,Genre_2
8443,1969,"Goodbye, Columbus",American,Larry Peerce,"Richard Benjamin, Ali MacGraw, Jack Klugman",romance,"https://en.wikipedia.org/wiki/Goodbye,_Columbu...",neil klugman (richard benjamin) is an intellig...,romance,romance
26621,2010,The Waiting Room,Bollywood,Maneej Premnath,"Raj Singh Chaudhary, Radhika Apte",thriller,https://en.wikipedia.org/wiki/The_Waiting_Room...,the story of the waiting room takes place in t...,thriller,thriller
30610,2005,Aayudham,Tamil,M. A. Murugesh,"Prashanth, Sneha",action,https://en.wikipedia.org/wiki/Aayudham_(2005_f...,shiva is a middle-class medical student in che...,action,action
30357,2000,Kakkai Siraginilae,Tamil,P. Vasu,"Parthiban, Preetha Vijayakumar, Lakshmi, K. Vi...",unknown,https://en.wikipedia.org/wiki/Kakkai_Siraginilae,vellisamy (parthiban) is an orphan who was bro...,unknown,unknown
26095,2001,Kasoor,Bollywood,Vikram Bhatt,"Lisa Ray, Aftab Shivdasani, Divya Dutta",thriller,https://en.wikipedia.org/wiki/Kasoor,"the film starts with the murder of priti, wife...",thriller,thriller


In [10]:
# создаем df жанрами, разделенными по столбцам
new_df = df_copy['Genre_2'].str.split('/', expand=True)
new_df.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12
4750,comedy,None,None,None,None,None,None,None,None,None,None,None,None
15132,adventure,comedy,drama,family,None,None,None,None,None,None,None,None,None
17980,drama,romance,,None,None,None,None,None,None,None,None,None,None
11984,adventure,None,None,None,None,None,None,None,None,None,None,None,None
2028,comedy,None,None,None,None,None,None,None,None,None,None,None,None


In [19]:
#new_df.to_excel('new_df.xlsx')

In [11]:
#добавляем первое слово жанра к df_copy (при перечислении жанров обычно основной указывается вначале)
df_copy['Genre_1st_word']=new_df[0]

In [12]:
#создадим словарь для обработки df_copy['Origin/Ethnicity']

dict_orig ={
    'American': 'American', 'British': 'British', 'Bollywood': 'Indian', 
    'Tamil': 'Indian', 'Telugu': 'Indian', 'Japanese': 'Asian', 'Malayalam': 'Indian',
    'Hong Kong': 'Asian', 'Canadian': 'Canadian', 'Australian': 'Australian', 'South_Korean': 'Asian',
    'Chinese': 'Asian', 'Kannada': 'Indian', 'Bengali': 'Indian', 'Russian': 'Russian', 'Marathi': 'Indian',
    'Filipino': 'Asian', 'Bangladeshi': 'Asian', 'Punjabi': 'Indian', 'Malaysian': 'Asian', 'Turkish':'Turkish',
    'Egyptian': 'Egyptian', 'Assamese': 'Indian', 'Maldivian': 'Asian'
}

In [13]:
#добавим в df_copy столбец с регионом происхождения
df_copy['Origin'] = df_copy['Origin/Ethnicity'].map(dict_orig)

In [14]:
# произведем некоторые замены в df_copy['Genre_1st_word']
df_copy['Genre_1st_word']=df_copy['Genre_1st_word'].str.replace('film', 'film_noir')
df_copy['Genre_1st_word']=df_copy['Genre_1st_word'].str.replace('romantic', 'romance')
df_copy['Genre_1st_word']=df_copy['Genre_1st_word'].str.replace('animation', 'animated')

In [15]:
#добавим столбец с количеством фильмов по жаннру
df_copy['n_films_by_genre']= df_copy.Genre_1st_word.map(df_copy['Genre_1st_word'].value_counts())

In [16]:
#приводим к нижнему регистру df_copy['Plot']
df_copy['Plot'] = df_copy['Plot'].str.lower()

In [17]:
#удаляем знаки препинания
df_copy['Plot'] = df['Plot'].str.replace('[^\w\s]','')

C:\Users\User\AppData\Local\Temp\ipykernel_8040\3468545993.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_copy['Plot'] = df['Plot'].str.replace('[^\w\s]','')


In [18]:
#жанров после обработки осталось 334
df_copy['Genre_1st_word'].value_counts()

drama            6888
unknown          6083
comedy           5638
action           2072
romance          1944
                 ... 
physiological       1
romance;            1
tapas               1
outlaw              1
travel              1
Name: Genre_1st_word, Length: 334, dtype: int64

In [19]:
# модели будем строить на df_copy3, попробуем обработать срез df с количсетвом жанров > 400
df_copy3 = df_copy[df_copy['n_films_by_genre'] > 400]

In [20]:
#удалим 'unknown' жанр
df_copy3 = df_copy3[df_copy3['Genre_1st_word'] != 'unknown']

In [21]:
df_copy3.columns

Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot', 'Genre_1', 'Genre_2', 'Genre_1st_word',
       'Origin', 'n_films_by_genre'],
      dtype='object')

In [22]:
#удалим ненужные колонки
df_copy3 = df_copy3.drop(['Release Year', 'Origin/Ethnicity',
                              'Director', 'Cast', 'Wiki Page',
                              'Genre_1', 'Genre_2', 'n_films_by_genre'
                             ], axis=1)

In [23]:
#проверим
df_copy3.columns

Index(['Title', 'Genre', 'Plot', 'Genre_1st_word', 'Origin'], dtype='object')

In [120]:
df_copy3['Genre_1st_word'].value_counts()

drama        6888
comedy       5638
action       2072
romance      1944
horror       1391
crime        1278
thriller     1061
western       924
animated      714
musical       708
adventure     610
science       495
war           408
family        402
Name: Genre_1st_word, dtype: int64

In [25]:
df_copy3['Origin'].value_counts()

American      14814
Indian         4507
British        2539
Asian          1479
Canadian        555
Australian      394
Russian         124
Egyptian         63
Turkish          58
Name: Origin, dtype: int64

# Предсказание жанра и региона фильма по краткому сценарию

# используем методы из модуля ML 

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn import tree

In [28]:
RANDOM_STATE = 42

# Предсказываем жанр

In [29]:
text_transformer = TfidfVectorizer(stop_words='english', 
                                   ngram_range=(1,1), 
                                   lowercase=True)
text = text_transformer.fit_transform(df_copy3['Plot'])
X_train, X_test, y_train, y_test = train_test_split(text, df_copy3['Genre_1st_word'], test_size=0.2, random_state=RANDOM_STATE)

In [30]:
X_train.shape

(19626, 145905)

In [31]:
clf2 = tree.DecisionTreeClassifier()
clf2.fit(X_train,y_train)

DecisionTreeClassifier()

In [34]:
%%time
pred2 = clf2.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test, pred2, average='weighted'))

Качество модели по метрике F1 0.31119034479764335
CPU times: total: 62.5 ms
Wall time: 67 ms


In [ ]:
#проверим, влияет ли набор 'stopwords' на качество модели

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
stop_eng = stopwords.words('english')

In [38]:
text_transformer = TfidfVectorizer(stop_words=stop_eng, 
                                   ngram_range=(1,1), 
                                   lowercase=True)
text = text_transformer.fit_transform(df_copy3['Plot'])
X_train, X_test, y_train, y_test = train_test_split(text, df_copy3['Genre_1st_word'], test_size=0.2, random_state=RANDOM_STATE)

In [39]:
clf2 = tree.DecisionTreeClassifier()
clf2.fit(X_train,y_train)

DecisionTreeClassifier()

In [40]:
%%time
pred2 = clf2.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test, pred2, average='weighted'))

Качество модели по метрике F1 0.3079334863587056
CPU times: total: 125 ms
Wall time: 116 ms


In [41]:
# вывод  -можно использовать базовый набор "stop_words='english"

# Предсказываем регион

In [43]:
df_copy4 = df_copy.copy()

In [45]:
#удалим ненужные колонки
df_copy4 = df_copy4.drop(['Release Year', 'Origin/Ethnicity',
                              'Director', 'Cast', 'Wiki Page',
                              'Genre_1', 'Genre_2', 'n_films_by_genre'
                             ], axis=1)

In [46]:
df_copy4.columns

Index(['Title', 'Genre', 'Plot', 'Genre_1st_word', 'Origin'], dtype='object')

In [48]:
text_transformer = TfidfVectorizer(stop_words='english', 
                                   ngram_range=(1,1), 
                                   lowercase=True)
text = text_transformer.fit_transform(df_copy4['Plot'])
X_train, X_test, y_train, y_test = train_test_split(text, df_copy4['Origin'], test_size=0.2, random_state=RANDOM_STATE)

In [49]:
clf2 = tree.DecisionTreeClassifier()
clf2.fit(X_train,y_train)

DecisionTreeClassifier()

In [50]:
%%time
pred2 = clf2.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test, pred2, average='weighted'))

Качество модели по метрике F1 0.6335158219958275
CPU times: total: 125 ms
Wall time: 139 ms


# Пробуем другой способ
https://medium.com/analytics-vidhya/predicting-movie-genres-using-nlp-46d70b97c67d

# Предсказываем жанр

In [108]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_copy3['Genre_1st_word'])

# transform target variable
y = multilabel_binarizer.transform(df_copy3['Genre_1st_word'])

In [109]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [110]:
# split dataset into training and validation set
xtrain, xval, ytrain, yval = train_test_split(df_copy3['Plot'], y, test_size=0.2, random_state=RANDOM_STATE)

In [111]:
# create TF-IDF features 
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain) 
xval_tfidf = tfidf_vectorizer.transform(xval)

In [112]:
from sklearn.linear_model import LogisticRegression 
# Binary Relevance 
from sklearn.multiclass import OneVsRestClassifier 
# Performance metric 
from sklearn.metrics import f1_score

In [113]:
lr = LogisticRegression() 
clf = OneVsRestClassifier(lr)

In [114]:
# fit model on train data 
clf.fit(xtrain_tfidf, ytrain)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


OneVsRestClassifier(estimator=LogisticRegression())

In [115]:
# make predictions for validation set 
y_pred = clf.predict(xval_tfidf)

In [116]:
# evaluate performance 
f1_score(yval, y_pred, average='weighted')

0.6716026494967324

In [117]:
#Для оценки модели произведем также расчет dummy_grid, F1 должна быть больше dummy_grid 

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [121]:
#Источник: https://github.com/Wonuabimbola/movie-genre-prediction/blob/main/predict_movie_genre_from_plot.ipynb

dummy_grid = {
    'strategy': ['most_frequent', 'stratified', 'prior', 'uniform', 'constant']
}
dummy = DummyClassifier(random_state = 42)
dummy_gs = GridSearchCV(dummy, param_grid = dummy_grid, cv = 5)
dummy_gs.fit(xtrain, ytrain)
print(dummy_gs.best_score_)

0.001171935585399963


C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\dummy.py", line 205, in fit
    raise ValueError(
ValueError: Constant target value has to be specified when the constant strategy is used.

  warnings.warn(some_fits_failed_message, FitFailedWarning)
C:\Users\User\anaconda3\lib\site-packages\sklearn\model

# Предсказываем регион

In [97]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_copy3['Origin'])

# transform target variable
y = multilabel_binarizer.transform(df_copy3['Origin'])

In [98]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [99]:
# split dataset into training and validation set
xtrain, xval, ytrain, yval = train_test_split(df_copy3['Plot'], y, test_size=0.2, random_state=RANDOM_STATE)

In [100]:
# create TF-IDF features 
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain) 
xval_tfidf = tfidf_vectorizer.transform(xval)

In [101]:
lr = LogisticRegression() 
clf = OneVsRestClassifier(lr)

In [102]:
# fit model on train data 
clf.fit(xtrain_tfidf, ytrain)

C:\Users\User\anaconda3\lib\site-packages\sklearn\multiclass.py:79: UserWarning: Label 13 is present in all training examples.
  warnings.warn(


OneVsRestClassifier(estimator=LogisticRegression())

In [103]:
# make predictions for validation set 
y_pred = clf.predict(xval_tfidf)

In [104]:
# evaluate performance 
f1_score(yval, y_pred, average='weighted')

0.8736109259730847

In [105]:
#Для оценки модели произведем также расчет dummy_grid, F1 должна быть больше dummy_grid 

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [107]:
dummy_grid = {
    'strategy': ['most_frequent', 'stratified', 'prior', 'uniform', 'constant']
}
dummy = DummyClassifier(random_state = 42)
dummy_gs = GridSearchCV(dummy, param_grid = dummy_grid, cv = 5)
dummy_gs.fit(xtrain, ytrain)
print(dummy_gs.best_score_)

0.6046058320976277


C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\User\anaconda3\lib\site-packages\sklearn\dummy.py", line 205, in fit
    raise ValueError(
ValueError: Constant target value has to be specified when the constant strategy is used.

  warnings.warn(some_fits_failed_message, FitFailedWarning)
C:\Users\User\anaconda3\lib\site-packages\sklearn\model

In [ ]:
# f1_score (0.8736109259730847) > dummy_grid (0.6046058320976277)
# f1 существенно выше, модель успешно предсказывает регион по сценарию